### Running Production Applications
* turn your interactive exploration into production applications with spark-submit
* spark-submit lets you send your application code to a cluster and launch it to execute there.
* You can do this with all of Sparks support cluster managers 
#####Structured APIs - Datasets, DataFrames, SQL
#####Low-level APIs - RDDs, Distributed Variables

Running sample Python version application on local machine, by running an app in the directory where you downloaded the spark
```./bin/spark-submit --master local ./examples/src/main/python/pi.py 10```
By changing the master argument, you can submit the same app to Cluster manager, YARN or MESOS.

####Datasets:Type-Safe Structured APIs
* Dataset API is not available in Python and R - as they are dynamically typed
* DataFrames area adistributed collection of objects of type Row, that can hold various types of tabular data
* Dataset API gives users the ability to assign a Java/Scala class to the records within a DataFrame and manipulate it as a collection of typed objects, similar to a Java ArrayList or Scala Seq.

In [4]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/FileStore/tables/2010_12_01-ec65d.csv")
#.csv("/FileStore/tables/2015_summary-ebaee.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema


In [5]:
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .show(5)


+----------+--------------------+------------------+
CustomerId| window| sum(total_cost)|
+----------+--------------------+------------------+
 13408.0|[2010-12-01 00:00...|1024.6800000000003|
 17460.0|[2010-12-01 00:00...| 19.9|
 15235.0|[2010-12-01 00:00...| 79.5|
 17905.0|[2010-12-01 00:00...|201.74999999999997|
 12583.0|[2010-12-01 00:00...| 855.86|
+----------+--------------------+------------------+
only showing top 5 rows

In [6]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/FileStore/tables/*.csv")


In [7]:
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")


In [8]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()


Out[ 20 ]: <pyspark.sql.streaming.StreamingQuery at 0x7fa856de0a50>

In [9]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)


+----------+------+---------------+
CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+

###Machine Learning and Advanced Analytics
* MLlib - allows for preprocessing, munging, training of models, and making predictions at scale on data
* Models trained in MLlib can be used to make predictions in Structured Streaming as well.
* MLlib algorithms require the data is represented as numerical values.

In [11]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)


In [12]:
trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")


In [13]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")


* StringIndexer - converts text to numerical values eg.Staurday to 6.
* OneHotEncoder - converts each of those numbers as their own column - to boolean values.
* All Machine learning algorithms in spark take input as a Vector type, which must be a set of numerical values.(VectorAssembler)

In [15]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")


In [16]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")


In [17]:
# set this process into a pipeline so that any future data we need to transform can go through the exact same process.
# preparing for training is a two step process 1. Fit 2. Transform
# Fit bcz StringIndexer needs to know how many unique values there are to be indexed.
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])


In [18]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)


In [19]:
transformedTraining = fittedPipeline.transform(trainDataFrame)


--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-3920840169979258> in <module> () 
 ----> 1 transformedTraining = fittedPipeline . transform ( trainDataFrame ) 

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 171 return self . copy ( params ) . _transform ( dataset ) 
 172 else : 
 --> 173 return self . _transform ( dataset ) 
 174 else : 
 175 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/pipeline.py in _transform (self, dataset) 
 260 def _transform ( self , dataset ) : 
 261 for t in self . stages : 
 --> 262 dataset = t . transform ( dataset ) 
 263 return dataset 
 264 

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 171 return self . copy ( params ) . _transform ( dataset ) 
 172 else : 
 --> 173 return self . _transform ( dataset ) 
 174 else : 
 175 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _transform (self, dataset) 
 303 def _transform ( self , dataset ) : 
 304 self . _transfer_params_to_java ( ) 
 --> 305 return DataFrame ( self . _java_obj . transform ( dataset . _jdf ) , dataset . sql_ctx ) 
 306 
 307 

 /databricks/spark/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1158 answer = self . gateway_client . send_command ( command ) 
 1159 return_value = get_return_value(
 -> 1160 answer, self.gateway_client, self.target_id, self.name)
 1161 
 1162 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 77 raise QueryExecutionException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 78 if s . startswith ( 'java.lang.IllegalArgumentException: ' ) : 
 ---> 79 raise IllegalArgumentException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 80 raise 
 81 return deco 

 IllegalArgumentException : u'requirement failed: The input column day_of_week_index should have at least two distinct values.'

In [20]:
# In Spark, training ML models is a two-phase process. First we initialize an untrained model(Algorithm - KMeans), and then train it (AlgotithmModel-kmModel)
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1L)


In [21]:
kmModel = kmeans.fit(transformedTraining)


In [22]:
transformedTest = fittedPipeline.transform(testDataFrame)


###Lower-Level APIs
* Spark include a number of lower-level primitives to allow for arbitrary Java and Python object manipulation via RDDs. Virtually everything in Spark is built on top of RDDs.
* DataFrames are compile down to these lower-level tools for convenient and extremely edfficient distributed execution.
* RDDs are lower level than DataFrames bcz they reveal physical execution characteristics (like partitions to end users)

In [24]:
# One thing you might use RDDs for is to parallelize raw data that you have stored in memory on the driver machine.
from pyspark.sql import Row

spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()
